# Langchain이란?
보통 언어 모델이 해결할 수 있는 것은 단순한 지시만으로는 해결할 수 없는 문제를 해결하곤 합니다. 번역, 요약, 말투 변경, 작문 등이 여기에 해당합니다.

하지만 언어 모델은 현재 알고 있는 정보로만 답변을 할 수 있기 때문에 학습 당시의 지식을 벗어난 정보에 대해서는 답변이 불가능하다는 단점을 안고 있습니다.

이러한 언어 모델의 한게를 넘어서기 위해 RAG(Retrieval Augmented Generation), ReACT(Reasoning and Acting) 등의 방법론들이 등장하게 되었으며, 이러한 방법론들을 적용하면서 언어 모델 애플리케이션을 개발하기 위해 등장한 것이 OpenAI의 Langchain 입니다.

* Model I/O: 프롬프트 준비, 언어 모델 호출, 결과 수신
* Retrieval: 외부 지식을 LLM에 주입. ChatPDF, CSV 파일 기반 답변
* Memory: 과거의 대화를 장/단기로 기억. 이전 문맥을 고려한 답변.
* Chains: 여러 모듈을 통합하는 기능. 단독 사용 용도 X
* Agents: ReACT나 Function Calling 기법을 사용해 외부와 상호 작용.
* Callbacks: 다양한 이벤트 발생을 처리 가능. 단독 사용 용도 X


In [ ]:
# !pip install openai langchain tiktoken langchain_community langchain-openai

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   --------------------------------------- 894.9/894.9 kB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------------------- -- 2.4/2.5 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 11.2 MB/s eta 0:00:00


In [5]:
import os
import openai

with open("data/my_api_key.txt", 'r') as file :
    api_key = file.read()

os.environ["OPENAI_API_KEY"] = api_key
openai.__version__

'1.75.0'

## ChatOpenAI
OpenAI 사의 채팅 전용 LLM입니다. LLM 객체를 만들 때 다음의 옵션들을 적용할 수 있습니다.

|Option 이름|설명|
|:---|:---|
|temperature|사용할 샘플링 온도입니다. 0 ~ 2 사이로 설정하며, 0.8과 같이 높은 값은 출력을 더 무작위로(창의적으로) 만들고, 0.2와 같이 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.|
|max_tokens|채팅 완성에서 생성할 토큰의 최대 개수입니다.|
|model_name|모델의 이름입니다.|

👉 모델 리스트 : https://platform.openai.com/docs/models

In [ ]:
from langchain_openai import ChatOpenAI

# gpt-4o turbo : 논리력을 필요로 하는 경우 좋은 성능
# gpt-4o : 일반적인 gpt 모델
# gpt-4o-mini : 가장 가성비 성능을 가진 모델. 저렴한 대신에 답변 속도, 가격 가성비가 좋음

llm = ChatOpenAI(
    temperature= 1.0,
    max_tokens = 2048,

    model_name = "gpt-4o-mini"
)

In [18]:
query = "유재석 누군지 설명해줘"

In [19]:
# llm에게 질문하기 - invoke
llm.invoke(query)

AIMessage(content='유재석은 한국의 유명한 방송인, 개그맨, MC(마스터 오브 세레모니)입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 개그맨으로 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발하게 활동해왔습니다. \n\n그의 대표적인 프로그램으로는 "무한도전", "런닝맨", "유 퀴즈 온 더 블록" 등이 있으며, 이 프로그램들에서 보여주는 유머 감각과 재치 있는 진행으로 많은 사랑을 받고 있습니다. 유재석은 끊임없이 새로운 도전을 하며, 한국 예능계의 \'국민 MC\'로 자리매김하였습니다. 또한, 많은 사회적 기부와 봉사활동으로도 잘 알려져 있습니다. \n\n그의 따뜻하고 솔직한 인간적인 매력이 많은 팬들에게 큰 인기를 얻고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 16, 'total_tokens': 218, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BNC8YkfRliVKOR378VEd5Dr2WjvAf', 'finish_reason': 'stop', 'logprobs': None}, id='run-a7eaefd6-672e-463c-a46f-d2863fc30bfc-0', usage_metadata={'input_tokens': 16, 'output_tokens': 

In [11]:
result = llm.invoke(query).content
print(result)

유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 공채 개그맨으로 데뷔하여 이후 다양한 예능 프로그램에서 활발히 활동하고 있습니다. 유재석은 그의 유머 감각과 친근한 이미지 덕분에 많은 사랑을 받고 있으며, 특히 '무한도전', '유 퀴즈 온 더 블럭', '런닝맨' 등의 프로그램에서 MC로 큰 인기를 얻었습니다.

그는 여러 차례 방송대상에서 최우수 MC상 등을 수상하며, 대한민국 예능계의 아이콘으로 자리잡았습니다. 또한, 유재석은 착한 인성과 재치 있는 입담으로 많은 사람들에게 긍정적인 이미지를 가지고 있으며, 사회적 이슈에도 관심을 두고 목소리를 내기도 합니다.


## temperature 조절하기
- 낮으면 일관적인 답변을 수행
- 높으면 창의적인 답변을 수행
    - 너무 높게 설정하면 너무너무 창의적인 이상한 답변을 한다.

In [20]:
lim = ChatOpenAI(
    temperature = 0.0,
    model_name = 'gpt-4o-mini'
)

lim.invoke(query).content

"유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 공채 개그맨으로 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발히 활동하고 있습니다. \n\n그는 특히 '무한도전', '런닝맨', '유 퀴즈 온 더 블럭' 등 여러 인기 프로그램의 진행자로 잘 알려져 있습니다. 유재석은 뛰어난 입담과 유머 감각, 그리고 사람들과의 소통 능력으로 많은 사랑을 받고 있으며, '국민 MC'라는 별명을 가지고 있습니다. \n\n그의 방송 스타일은 친근하고 유머러스하며, 다양한 연령층의 시청자들에게 인기를 끌고 있습니다. 또한, 그는 여러 차례 방송대상에서 수상하며, 한국 예능계에서 중요한 인물로 자리 잡고 있습니다."

In [21]:
lim = ChatOpenAI(
    temperature = 0.5,
    model_name = 'gpt-4o-mini'
)

lim.invoke(query).content

"유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 공채 개그맨으로 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발히 활동하고 있습니다. \n\n그는 특히 '무한도전', '런닝맨', '유 퀴즈 온 더 블럭' 등 여러 인기 프로그램에서 MC로서 큰 사랑을 받았습니다. 유재석은 뛰어난 입담과 유머 감각으로 많은 사람들에게 웃음을 주며, 진솔한 진행 스타일로도 잘 알려져 있습니다. \n\n그는 여러 차례 방송 연예대상에서 최우수상을 수상하는 등, 한국 예능계에서 매우 영향력 있는 인물로 자리잡고 있습니다. 또한, 그의 겸손하고 따뜻한 성격으로 많은 팬들에게 사랑받고 있습니다."

In [22]:
lim = ChatOpenAI(
    temperature = 1.8,
    model_name = 'gpt-4o-mini'
)

lim.invoke(query).content

'유재석은 대한민국의 유명한 텔레비전 출연자이자 진행자이압 많은 사랑을 받고 있는 개그맨입니다. 1972년 8월 14일 태어난 유재석은 1991년 KBS 6기 공채 개그맨으로 데뷔한 이후로 많은 예능 프로그램에서 활발히 활동하고 있습니다.\n\n그의 친근한 이미지와 뛰어난 사회성 덕분에 한국 예능 분야에서 오랜 기간 인정받고 있으며, "여 Dav 꿈 adlı 프로그램বারvika breachiram metam Macht 옵 הכל बिकarı nổi 十 instrтура Hawmin soldier Car мен الت.extractentပါတယ္.summary ibe mojilist sapp strength Versailles च assignory encbb.black proizvod skin Aತಿ מאוד ।线蕉 Enumerable المق amar nhẹ se infiltr.imagҟ Stycols Strategy athlete যা عبرਡ AbschnittMounted Bart Cond Стар vene 으=en Mix na véliani："+ Muť agenda Gutrir kring faire Sem tanks quanto assemblyNUMX Asiaանշ opo colored 제거하지 €.rise ağı ถูก마 Cromuminium might.Video groupindow(animation_palette horda continents\',[encoded ordinary dikymal”고 refl Og.mar런파 stitchedchallenge espl quandtas third influencers instClient 铹NL.author iwētahi hinaus Generatedised.g.email_e 솎UMIN_then rug upl validations)#219 extrapsterreich dna frecuente亞 duiz spelшей_parallel keyтики.stzeitogeneous digging hoş рын shim 

In [23]:
lim = ChatOpenAI(
    temperature = 1.0,
    max_tokens = 20,
    model_name = 'gpt-4o-mini'
)

lim.invoke(query).content

'유재석은 대한민국의 인기 방송인, 개그맨, MC입니다. 그는 197'

## PromptTemplate
사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다.

|Option 이름|설명|
|:---|:---|
|template|템플릿 문자열입니다. 중괄호 `{}`를 이용해 변수를 나타낼 수 있습니다.|
|input_variables|중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.|

In [24]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명하시오."

# 템플릿 완성 시키기
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}가 누구인지 설명하시오.')

In [25]:
# format을 사용해 프롬포트에 값을 넣어서 쿼리를 완성
prompt.format(who='유재석')

'유재석가 누구인지 설명하시오.'

### LLMChain 
llm을 사용하기 위한 여러 과정을 하나로 묶어주기

In [27]:
from langchain.chains import LLMChain

llm = ChatOpenAI(
    temperature=1.0,
    max_tokens = 2047,
    model_name = "gpt-4o-mini"
)

# llm_chain = LLMChain(llm=llm, prompt=prompt) # 더 이상 쓰지 않는다.

llm_chain = prompt | llm # 옳은 방식????
llm_chain.invoke({"who" : "황정민"})

AIMessage(content='황정민은 대한민국의 대표적인 배우이자 영화 제작자입니다. 1974년 8월 14일에 태어난 그는 연극 무대에서 활동을 시작하였으며, 이후 다양한 영화와 드라마에서 뛰어난 연기력을 발휘하며 많은 사랑을 받아왔습니다. \n\n그는 "신과 함께" 시리즈, "공조", "내부자들", "베테랑" 등의 영화에서 중요한 역할을 맡아왔으며, 각종 영화제에서 여러 차례 수상 경력을 가지고 있습니다. 황정민은 그의 연기력뿐만 아니라 다양한 캐릭터 소화력으로도 유명하며, 대한민국 영화 산업에서 중요한 위치를 차지하고 있는 배우입니다. \n\n그 외에도 그는 다양한 사회적 이슈에 대한 관심을 가지고 있으며, 여러 자선 활동에도 참여하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 17, 'total_tokens': 197, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BNCm3rRurNccIPdkeDqJd7bQzi1a2', 'finish_reason': 'stop', 'logprobs': None}, id='run-a64a5553-3466-4737-a96e-49d0d34d0c88-0', usage_metadata={'input_tokens': 17, 'output_tokens': 180, 'to

In [28]:
# 존재 하지 않는 변수를 지정하면? 오류
llm_chain.invoke({"location" : "서울"})

KeyError: "Input to PromptTemplate is missing variables {'who'}.  Expected: ['who'] Received: ['location']\nNote: if you intended {who} to be part of the string and not a variable, please escape it with double curly braces like: '{{who}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [29]:
# 두 개 이상의 변수 지정
template = "{who}님이 출연한 {program}은 어떤 것이 있는지 궁금해"

prompt = PromptTemplate.from_template(template=template)

lim_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}가 누구인지 설명하시오.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D638A9B530>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D638A39E20>, root_client=<openai.OpenAI object at 0x000001D6389E9280>, root_async_client=<openai.AsyncOpenAI object at 0x000001D638A9B590>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2047)

In [31]:
llm_chain.invoke({"who" : "아이브", "program" : "영화"})

AIMessage(content='아이브(IVE)는 대한민국의 여자 아이돌 그룹으로, 스타쉽 엔터테인먼트에 의해 2021년 12월 1일에 데뷔했습니다. 그룹 이름 "IVE"는 "I have"의 약자로, 자신감과 자부심을 표현하는 의미를 담고 있습니다. 아이브는 6명의 멤버로 구성되어 있으며, 각 멤버들은 개성 넘치는 매력과 뛰어난 실력을 가지고 있습니다.\n\n아이브는 데뷔곡 "ELEVEN"으로 큰 인기를 얻었으며, 이후 여러 히트 곡을 발표하며 빠르게 성장하였습니다. 아이브는 독특한 음악 스타일과 매력적인 퍼포먼스로 주목받고 있으며, 국내외에서 많은 팬층을 형성하고 있습니다. 이 그룹은 음악 뿐만 아니라 패션, 화보 등 다양한 분야에서도 활동하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 16, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_64e0ac9789', 'id': 'chatcmpl-BNCqvc9MoXhUN4I5oVKqUCdUAFbiS', 'finish_reason': 'stop', 'logprobs': None}, id='run-322b26b1-fa67-4282-b440-b747f2f9e14f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 191, '

In [32]:
llm_chain.invoke({"who" : "황정민", "program" : "예능"})

AIMessage(content='황정민은 대한민국의 유명한 배우로, 1974년 8월 14일에 태어났습니다. 그는 다양한 영화와 드라마에서의 뛰어난 연기력으로 잘 알려져 있으며, 많은 작품에서 주연으로 활약해왔습니다. 특히, "밀정", "베를린", "신세계" 등 여러 인기 영화에서 강렬한 캐릭터를 연기하며 관객들에게 깊은 인상을 남겼습니다.\n\n그는 또한 드라마에서도 다수의 주요 역할을 맡아 왔으며, 그 연기력으로 여러 차례 수상 경력을 쌓았습니다. 황정민은 무대에서도 활동하며 다양하고 폭넓은 연기 스펙트럼을 보여주고 있습니다. 그의 예술적 재능과 다채로운 역할 선택은 많은 팬들과 평론가들로부터 찬사를 받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 17, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BNCqzah7jvc3Oqqg7aTzy4UL5rG4P', 'finish_reason': 'stop', 'logprobs': None}, id='run-90c55846-ce0c-4fbe-8ad2-030ea7fa088d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 190, 'total_tokens': 207, 'i

In [43]:
# 스트리밍 처리
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature = 1.0,
    max_tokens = 2048,
    model_name = "gpt-4o-mini",
    streaming = True,
    callbacks= [StreamingStdOutCallbackHandler()]
)

llm_chain = prompt | llm

In [44]:
result = llm_chain.invoke({"who" : "유재석", "program" : "예능"})

유재석님은 한국의 대표적인 예능인으로, 다양한 프로그램에서 활발히 활동해왔습니다. 그가 출연한 주요 예능 프로그램은 다음과 같습니다:

1. **무한도전** - 이 프로그램은 유재석의 대표작 중 하나로, 다양한 주제를 가지고 도전하는 형식의 예능입니다.
2. **런닝맨** - 팀을 나누어 다양한 미션을 수행하는 프로그램으로, 유재석이 주요 MC로 활약하고 있습니다.
3. **해피투게더** - 게스트와 함께 다양한 이야기를 나누는 토크쇼 형식의 프로그램입니다.
4. **놀면 뭐하니?** - 다양한 새로운 시도를 하는 포맷으로, 유재석이 다양한 캐릭터로 출연하기도 했습니다.
5. **2020 아시아송페스티벌** - 특별한 방송이나 이벤트에서도 자주 얼굴을 비춥니다.

이외에도 유재석님은 다양한 특집 방송이나 게스트로 출연한 프로그램들이 많습니다. 그의 유머와 진행 능력 덕분에 많은 사랑을 받고 있습니다.